# Exercise 3: Two-Strain Model (R Version)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ngozzi/tech-transfer-epdemix/blob/main/sessions/session-4/exercises/r-colab/exercise_3_two_strains.ipynb)

**Objective:** Model the emergence of a more transmissible variant using a multi-strain SIR model, using R via the `reticulate` package.

**Skills practiced:**
- Designing complex compartmental structures
- Using `override_parameter` for delayed strain emergence
- Interpreting multi-strain dynamics

In [ ]:
!pip install epydemix
%load_ext rpy2.ipython

In [ ]:
%%R
if (!require("reticulate", quietly = TRUE)) {
  install.packages("reticulate")
}
library(reticulate)
use_python("/usr/bin/python3", required = TRUE)

## Model Structure

We model two strains where strain 2 is more transmissible:

**Compartments:** S, I1, I2, R1, R2

**Transitions:**
- S + I1 → 2I1 (rate β) — infection with strain 1
- S + I2 → 2I2 (rate β·ψ) — infection with strain 2 (ψ > 1)
- R1 + I2 → I2 + R2 (rate β·ψ·γ) — reinfection of R1 by strain 2
- I1 → R1 (rate μ) — recovery from strain 1
- I2 → R2 (rate μ) — recovery from strain 2

**Parameters:**
- β = 0.20 (base transmission rate)
- μ = 0.1 (recovery rate)
- ψ = 1.5 (strain 2 is 50% more transmissible)
- γ = 0.4 (R1 individuals have 60% protection against strain 2)

## Task 1: Implement the Two-Strain Model

In [ ]:
%%R
# Import epydemix
epydemix <- import("epydemix")
EpiModel <- epydemix$EpiModel
builtins <- import_builtins()
np <- import("numpy")
viz <- import("epydemix.visualization")
plot_quantiles <- viz$plot_quantiles

# Function to create a two-strain model
create_two_strain_model <- function(psi = 1.5, gamma_val = 0.4, beta = 0.20, mu = 0.1) {
  # TODO: Create the model with compartments S, I1, I2, R1, R2
  model <- ...
  
  # TODO: Add transitions
  # S + I1 -> 2I1
  # Hint: params_S_I1 <- builtins$tuple(list("beta", "I1"))
  
  # S + I2 -> 2I2 (more transmissible)
  # Hint: params_S_I2 <- builtins$tuple(list("beta*psi", "I2"))
  
  # R1 + I2 -> I2 + R2 (reinfection)
  # Hint: params_R1_I2 <- builtins$tuple(list("beta*psi*gamma", "I2"))
  
  # I1 -> R1
  
  # I2 -> R2
  
  # TODO: Set parameters (beta, psi, gamma, mu1, mu2)
  
  return(model)
}

# Create the model
model <- create_two_strain_model(psi = 1.5, gamma_val = 0.4)
model

## Task 2: Delay Strain 2 Emergence

Use `override_parameter` to set ψ=0 and μ2=0 for the first 30 days, effectively turning off strain 2.

**Hint:**
```r
model$override_parameter(
  start_date = "2025-01-01",
  end_date = "2025-01-31",
  parameter_name = "psi",
  value = 0.0
)
```

In [ ]:
%%R
# TODO: Override psi to 0 for the first 30 days

# TODO: Override mu2 to 0 for the first 30 days

## Task 3: Simulate and Visualize

Run simulations for 6 months with initial conditions seeding both strains.

In [ ]:
%%R
# Get default population Nk
Nk_val <- py_to_r(model$population$Nk)

# TODO: Set up initial conditions
# Use reticulate::dict() and np$array() to create:
# - S: population - 20
# - I1: 10 (strain 1 infected)
# - I2: 10 (strain 2 infected, but dormant until day 30)
# - R1: 0
# - R2: 0
initial_conditions <- ...

# TODO: Run simulations for 6 months
results <- ...

In [ ]:
%%R
# TODO: Plot both strains (I1_total and I2_total)
df_quantiles <- ...

# Use plot_quantiles with ax parameter to overlay plots

## Task 4: Experiments

### Experiment A: Effect of Emergence Timing

What happens if strain 2 emerges earlier (day 15) vs. later (day 45)?

In [ ]:
%%R
run_with_emergence_day <- function(emergence_day) {
  m <- create_two_strain_model(psi = 1.5, gamma_val = 0.4)
  
  emergence_date <- as.character(as.Date("2025-01-01") + emergence_day)
  
  # TODO: Override parameters until emergence
  
  # TODO: Set up initial conditions
  
  # TODO: Run simulations
  res <- ...
  
  list(results = res, emergence_date = emergence_date)
}

# TODO: Run with different emergence times (day 15, 30, 45)
res_early <- ...
res_mid <- ...
res_late <- ...

In [ ]:
%%R
# TODO: Visualize early emergence scenario

In [ ]:
%%R
# TODO: Visualize mid emergence scenario

In [ ]:
%%R
# TODO: Visualize late emergence scenario

### Experiment B: Effect of Transmissibility Advantage

What if strain 2 is only 20% more transmissible (ψ = 1.2) vs 100% more transmissible (ψ = 2.0)?

In [ ]:
%%R
run_with_psi <- function(psi_value) {
  # TODO: Create model with specified psi
  # TODO: Override parameters for first 30 days
  # TODO: Set up initial conditions
  # TODO: Run simulations
  ...
}

# TODO: Run with different psi values (1.2, 1.5, 2.0)
results_psi_low <- ...
results_psi_mid <- ...
results_psi_high <- ...

In [ ]:
%%R
# TODO: Visualize psi = 1.2 scenario

In [ ]:
%%R
# TODO: Visualize psi = 1.5 scenario

In [ ]:
%%R
# TODO: Visualize psi = 2.0 scenario

In [ ]:
%%R
# Optional: Compute dominance day
compute_dominance_day <- function(results) {
  traj <- results$get_stacked_compartments()
  I1_median <- apply(py_to_r(traj["I1_total"])$I1_total, 2, median)
  I2_median <- apply(py_to_r(traj["I2_total"])$I2_total, 2, median)
  
  crossover <- which(I2_median > I1_median & I2_median > 100)
  if (length(crossover) > 0) return(crossover[1])
  return(NA)
}

# TODO: Print days until dominance for each psi value

## Discussion

*Write your observations here:*

1. **Emergence timing:** How does the timing of strain 2 emergence affect the dynamics? What happens when strain 1 has already infected a large portion of the population?

2. **Transmissibility advantage (ψ):** How does the transmissibility advantage affect the speed of strain replacement? At what ψ value does strain 2 fail to dominate?

3. **Cross-immunity (γ):** With γ = 0.4 (60% protection), how does this partial immunity affect the dynamics? What would happen with complete cross-protection (γ = 0)?

4. **Real-world implications:** What do these results suggest about variant emergence in real pandemics (e.g., SARS-CoV-2 Alpha, Delta, Omicron)?